In [1]:
import numpy as np
import pandas as pd
import tqdm
from sklearn import tree
from kmodes.kmodes import KModes
from sklearn.cluster import SpectralClustering
from collections import Counter
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.cluster import AffinityPropagation
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import math 
import sklearn
import scipy.special as sp
from tqdm import tqdm
import SimpSOM as sps
from sklearn.decomposition import PCA
from sklearn.decomposition import KernelPCA
from sklearn.decomposition import FastICA
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
%matplotlib notebook

df=[]
df1=[]
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/b"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/f2"+str(i)+".csv"))

#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/ec"+str(i)+".csv"))
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/n"+str(i)+".csv"))
    
#for i in range(7):
#    df.append(pd.read_csv("/home/franc/franc/matches/d2"+str(i)+".csv"))
    
#for i in range(6):
#    df.append(pd.read_csv("/home/franc/franc/matches/d1"+str(i)+".csv"))    
df1.append(pd.read_csv("/home/franc/franc/matches/SWE.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/DNK.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/CHN.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/ARG.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/NOR.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/JPN.csv"))#Doesnt work well
#df1.append(pd.read_csv("/home/franc/franc/matches/AUT.csv"))
#df1.append(pd.read_csv("/home/franc/franc/matches/POL.csv"))

#df=pd.concat(df,sort=False)
df1=pd.concat(df1,sort=False)
rat_dic={}
fet_set=[]
tar_set=[]

In [2]:
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 
  
  
# Function to calculate Elo rating 
# K is a constant. 
# d determines whether 
# Player A wins or Player B.  
def EloRating(Ra, Rb, K, d): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = nprob(Ra, Rb) 
    Pb = 1-Pa
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if (d == 1) : 
        Ra = Ra + K * (1 - Pa) 
        Rb = Rb + K * (0 - Pb) 
      
  
    # Case -2 When Player B wins 
    # Updating the Elo Ratings 
    elif (d==0.5) : 
        Ra = Ra + K * (0.5 - Pa) 
        Rb = Rb + K * (0.5 - Pb) 
      
    else : 
        Ra = Ra + K * (0 - Pa) 
        Rb = Rb + K * (1 - Pb) 
      
    return Ra,Rb
    print("Updated Ratings:-") 
    print("Ra =", round(Ra, 6)," Rb =", round(Rb, 6))

def Probabilityad(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * ((rating1) - rating2) / 400)) 

def nprob(rating1, rating2): 
  
    return 1/2*(1 + sp.erf((rating1-rating2)/(300*math.sqrt(2)))) 

def spreadelo(Ra, Rb, K, ga,gb,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga-gb > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def adelo(Ra, Rb, K, g,n): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if g > n:
        Ra=Ra+K*(1-Pa)
        Rb=Rb+K*(0-Pb)
    else:
        Ra=Ra+K*(0-Pa)
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

def kgvelo(Ra, Rb, K, ga,gb): 
   
      
    # To calculate the Winning 
    # Probability of Player B 
    Pa = nprob(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pb = 1-Pa 
  
    # Case -1 When Player A wins 
    # Updating the Elo Ratings 
    if ga==0:
        Ra=Ra+K*(0-Pa)
    else:
        Ra=Ra+K*(1-Pa)
        
    if gb==0:
        Rb=Rb+K*(0-Pb)
    else:
        Rb=Rb+K*(1-Pb)
      
    return Ra,Rb

In [ ]:
df['Date']=pd.to_datetime(df['Date']
                              ,infer_datetime_format=True)
last_two=df[(df["Date"].dt.year>2012) & (df["Date"].dt.year<2018)]
for i in df.HomeTeam.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test=df[(df["Date"].dt.year>=2014) & (df["Date"].dt.year<2019)]

for i in range(len(last_two)):
    rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr"],rat_dic[last_two.iloc[i,3]]["tr"],60,last_two.iloc[i,4],last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr1"],rat_dic[last_two.iloc[i,3]]["tr-1"],60,last_two.iloc[i,4],last_two.iloc[i,5],1)
    rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,2]]["tr2"],rat_dic[last_two.iloc[i,3]]["tr-2"],60,last_two.iloc[i,4],last_two.iloc[i,5],2)
    rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr1"],rat_dic[last_two.iloc[i,2]]["tr-1"],60,last_two.iloc[i,5],last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"]=spreadelo(rat_dic[last_two.iloc[i,3]]["tr2"],rat_dic[last_two.iloc[i,2]]["tr-2"],60,last_two.iloc[i,5],last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"]=adelo(rat_dic[last_two.iloc[i,2]]["ar"],rat_dic[last_two.iloc[i,3]]["dr"],60,last_two.iloc[i,4],0)
    rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"]=adelo(rat_dic[last_two.iloc[i,3]]["ar"],rat_dic[last_two.iloc[i,2]]["dr"],60,last_two.iloc[i,5],0)
    rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"]=adelo(rat_dic[last_two.iloc[i,2]]["ar1"],rat_dic[last_two.iloc[i,3]]["dr1"],60,last_two.iloc[i,4],1)
    rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"]=adelo(rat_dic[last_two.iloc[i,3]]["ar1"],rat_dic[last_two.iloc[i,2]]["dr1"],60,last_two.iloc[i,5],1)    
    rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"]=adelo(rat_dic[last_two.iloc[i,2]]["ar2"],rat_dic[last_two.iloc[i,3]]["dr2"],60,last_two.iloc[i,4],2)
    rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"]=adelo(rat_dic[last_two.iloc[i,3]]["ar2"],rat_dic[last_two.iloc[i,2]]["dr2"],60,last_two.iloc[i,5],2)     
    rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"]=kgvelo(rat_dic[last_two.iloc[i,2]]["kgv"],rat_dic[last_two.iloc[i,3]]["kgv"],60,last_two.iloc[i,4],last_two.iloc[i,5])
    rat_dic[last_two.iloc[i,2]]["nm"]+=1
    rat_dic[last_two.iloc[i,3]]["nm"]+=1
    

#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr"],rat_dic[test.iloc[t,3]]["tr"]))   
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr1"],rat_dic[test.iloc[t,3]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["tr2"],rat_dic[test.iloc[t,3]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr1"],rat_dic[test.iloc[t,2]]["tr-1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["tr2"],rat_dic[test.iloc[t,2]]["tr-2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar"],rat_dic[test.iloc[t,3]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar"],rat_dic[test.iloc[t,2]]["dr"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar1"],rat_dic[test.iloc[t,3]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar1"],rat_dic[test.iloc[t,2]]["dr1"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["ar2"],rat_dic[test.iloc[t,3]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,3]]["ar2"],rat_dic[test.iloc[t,2]]["dr2"]))
    tmp.append(nprob(rat_dic[test.iloc[t,2]]["kgv"],rat_dic[test.iloc[t,3]]["kgv"]))
    tmp1.append(test.iloc[t,4])
    tmp1.append(test.iloc[t,5])
    tmp1.append(test.iloc[t,4]+test.iloc[t,5])
    tmp1.append(test.iloc[t,4]-test.iloc[t,5])
    tmp1.append((test.iloc[t,4]!=0) and (test.iloc[t,5]!=0))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>1.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>2.5))
    tmp1.append(1*((test.iloc[t,4]+test.iloc[t,5])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))

In [3]:
df1['Date']=pd.to_datetime(df1['Date']
                              ,infer_datetime_format=True)
last_two1=df1[(df1["Date"].dt.year>2012) & (df1["Date"].dt.year<2018)]
for i in df1.Home.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

for i in df1.Away.unique():
    rat_dic[i]={"tr":1500,"tr1":1500,"tr-1":1500,"tr2":1500,"tr-2":1500,"ar":1500,"ar1":1500,"ar2":1500,"dr":1500,"dr1":1500,"dr2":1500,"kgv":1500,"nm":0}

test1=df1[(df1["Date"].dt.year>=2014) & (df1["Date"].dt.year<2019)]

for i in range(len(last_two1)):
    rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr"],rat_dic[last_two1.iloc[i,6]]["tr"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr1"],rat_dic[last_two1.iloc[i,6]]["tr-1"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],1)
    rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,5]]["tr2"],rat_dic[last_two1.iloc[i,6]]["tr-2"],60,last_two1.iloc[i,7],last_two1.iloc[i,8],2)
    rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr1"],rat_dic[last_two1.iloc[i,5]]["tr-1"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"]=spreadelo(rat_dic[last_two1.iloc[i,6]]["tr2"],rat_dic[last_two1.iloc[i,5]]["tr-2"],60,last_two1.iloc[i,8],last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar"],rat_dic[last_two1.iloc[i,6]]["dr"],60,last_two1.iloc[i,7],0)
    rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar"],rat_dic[last_two1.iloc[i,5]]["dr"],60,last_two1.iloc[i,8],0)
    rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar1"],rat_dic[last_two1.iloc[i,6]]["dr1"],60,last_two1.iloc[i,7],1)
    rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar1"],rat_dic[last_two1.iloc[i,5]]["dr1"],60,last_two1.iloc[i,8],1)    
    rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,5]]["ar2"],rat_dic[last_two1.iloc[i,6]]["dr2"],60,last_two1.iloc[i,7],2)
    rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"]=adelo(rat_dic[last_two1.iloc[i,6]]["ar2"],rat_dic[last_two1.iloc[i,5]]["dr2"],60,last_two1.iloc[i,8],2)     
    rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"]=kgvelo(rat_dic[last_two1.iloc[i,5]]["kgv"],rat_dic[last_two1.iloc[i,6]]["kgv"],60,last_two1.iloc[i,7],last_two1.iloc[i,8])
    rat_dic[last_two1.iloc[i,5]]["nm"]+=1
    rat_dic[last_two1.iloc[i,6]]["nm"]+=1
    
#tar_set ga,gb,tg,s,kgv,oo,to,tho
for t in range(len(test1)):
    tmp=[]
    tmp1=[]
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr"],rat_dic[test1.iloc[t,6]]["tr"]))   
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr1"],rat_dic[test1.iloc[t,6]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["tr2"],rat_dic[test1.iloc[t,6]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr1"],rat_dic[test1.iloc[t,5]]["tr-1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["tr2"],rat_dic[test1.iloc[t,5]]["tr-2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar"],rat_dic[test1.iloc[t,6]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar"],rat_dic[test1.iloc[t,5]]["dr"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar1"],rat_dic[test1.iloc[t,6]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar1"],rat_dic[test1.iloc[t,5]]["dr1"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["ar2"],rat_dic[test1.iloc[t,6]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,6]]["ar2"],rat_dic[test1.iloc[t,5]]["dr2"]))
    tmp.append(nprob(rat_dic[test1.iloc[t,5]]["kgv"],rat_dic[test1.iloc[t,6]]["kgv"]))
    tmp1.append(test1.iloc[t,7])
    tmp1.append(test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]+test1.iloc[t,8])
    tmp1.append(test1.iloc[t,7]-test1.iloc[t,8])
    tmp1.append((test1.iloc[t,7]!=0) and (test1.iloc[t,8]!=0))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>1.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>2.5))
    tmp1.append(1*((test1.iloc[t,7]+test1.iloc[t,8])>3.5))
    fet_set.append(np.array(tmp))
    tar_set.append(np.array(tmp1))



In [ ]:
n=40
for i in range(n):
    t1=df[(df["Date"].dt.year==2019) ].iloc[i,2]
    t2=df[(df["Date"].dt.year==2019) ].iloc[i,3]
    tmp=[]
    
    tmp1=[]
#t1="Anderlecht"
#t2="Oostende"
    tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
    tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
    tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
    tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
    tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
    tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
    tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
    tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
    tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
    tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
    tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
    tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
    fet_set.append(tmp)
    tmp1.append(test.iloc[i,4])
    tmp1.append(test.iloc[i,5])
    tmp1.append(test.iloc[i,4]+test.iloc[i,5])
    tmp1.append(test.iloc[i,4]-test.iloc[i,5])
    tmp1.append((test.iloc[i,4]!=0) and (test.iloc[i,5]!=0))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>1.5))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>2.5))
    tmp1.append(1*((test.iloc[i,4]+test.iloc[i,5])>3.5))
    tar_set.append(np.array(tmp1))
    print(t1,t2,tmp1,i)

In [4]:
n=40
for i in range(n):
    t1=df1[(df1["Date"].dt.year==2019) ].iloc[i,5]
    t2=df1[(df1["Date"].dt.year==2019) ].iloc[i,6]
    tmp=[]
    
    tmp1=[]
#t1="Anderlecht"
#t2="Oostende"
    tmp.append(nprob(rat_dic[t1]["tr"],rat_dic[t2]["tr"]))   
    tmp.append(nprob(rat_dic[t1]["tr1"],rat_dic[t2]["tr-1"]))
    tmp.append(nprob(rat_dic[t1]["tr2"],rat_dic[t2]["tr-2"]))
    tmp.append(nprob(rat_dic[t2]["tr1"],rat_dic[t1]["tr-1"]))
    tmp.append(nprob(rat_dic[t2]["tr2"],rat_dic[t1]["tr-2"]))
    tmp.append(nprob(rat_dic[t1]["ar"],rat_dic[t2]["dr"]))
    tmp.append(nprob(rat_dic[t2]["ar"],rat_dic[t1]["dr"]))
    tmp.append(nprob(rat_dic[t1]["ar1"],rat_dic[t2]["dr1"]))
    tmp.append(nprob(rat_dic[t2]["ar1"],rat_dic[t1]["dr1"]))
    tmp.append(nprob(rat_dic[t1]["ar2"],rat_dic[t2]["dr2"]))
    tmp.append(nprob(rat_dic[t2]["ar2"],rat_dic[t1]["dr2"]))
    tmp.append(nprob(rat_dic[t1]["kgv"],rat_dic[t2]["kgv"]))
    fet_set.append(tmp)
    tmp1.append(test1.iloc[i,7])
    tmp1.append(test1.iloc[i,8])
    tmp1.append(test1.iloc[i,7]+test1.iloc[i,8])
    tmp1.append(test1.iloc[i,7]-test1.iloc[i,8])
    tmp1.append((test1.iloc[i,7]!=0) and (test1.iloc[i,8]!=0))
    tmp1.append(1*((test1.iloc[i,7]+test1.iloc[i,8])>1.5))
    tmp1.append(1*((test1.iloc[i,7]+test1.iloc[i,8])>2.5))
    tmp1.append(1*((test1.iloc[i,7]+test1.iloc[i,8])>3.5))
    tar_set.append(np.array(tmp1))
    print(t1,t2,tmp1,i)

AIK Ostersunds [1, 2, 3, -1, True, 1, 1, 0] 0
Falkenbergs Orebro [1, 1, 2, 0, True, 1, 0, 0] 1
Kalmar Sirius [2, 2, 4, 0, True, 1, 1, 1] 2
AFC Eskilstuna Goteborg [1, 2, 3, -1, True, 1, 1, 0] 3
Helsingborg Norrkoping [3, 0, 3, 3, False, 1, 1, 0] 4
Djurgarden Sundsvall [2, 1, 3, 1, True, 1, 1, 0] 5
Elfsborg Hammarby [2, 0, 2, 2, False, 1, 0, 0] 6
Malmo FF Hacken [0, 2, 2, -2, False, 1, 0, 0] 7
Hacken Helsingborg [3, 1, 4, 2, True, 1, 1, 1] 8
Sundsvall Malmo FF [1, 0, 1, 1, False, 0, 0, 0] 9
Goteborg Elfsborg [3, 0, 3, 3, False, 1, 1, 0] 10
Ostersunds Falkenbergs [1, 2, 3, -1, True, 1, 1, 0] 11
Hammarby Kalmar [2, 0, 2, 2, False, 1, 0, 0] 12
Norrkoping AIK [2, 2, 4, 0, True, 1, 1, 1] 13
Orebro Djurgarden [0, 3, 3, -3, False, 1, 1, 0] 14
Sirius AFC Eskilstuna [1, 0, 1, 1, False, 0, 0, 0] 15
AFC Eskilstuna Orebro [0, 2, 2, -2, False, 1, 0, 0] 16
Falkenbergs Hacken [2, 0, 2, 2, False, 1, 0, 0] 17
Kalmar Norrkoping [1, 0, 1, 1, False, 0, 0, 0] 18
AIK Sirius [1, 1, 2, 0, True, 1, 0, 0] 19
Elf

In [10]:
fet_set=np.array(fet_set)
net = sps.somNet(20, 20, fet_set, PBC=True)
net.train(0.01, 10000)

Periodic Boundary Conditions active.
The weights will be initialised randomly.
Training SOM... done! 


SystemExit: Error: unkown clustering algorithm kmeans

/home/franc/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3275: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [11]:
c=net.cluster(fet_set, type='KMeans',numcl=200)

<Figure size 432x288 with 0 Axes>

In [13]:
c

[[164, 276, 824],
 [148,
  201,
  238,
  279,
  295,
  351,
  493,
  565,
  586,
  613,
  784,
  817,
  834,
  896,
  1151,
  1203],
 [171, 190, 200, 293, 670],
 [452, 499],
 [749, 1006, 1012, 1038],
 [127, 255, 578, 737, 1099, 1239],
 [125, 440, 497, 966],
 [384, 628, 953, 1189],
 [277, 564, 723, 865, 930, 1037, 1148],
 [161, 480, 484, 801, 1085],
 [603, 772, 1094],
 [49, 208, 377, 629],
 [358, 464, 486, 519, 767, 877, 1060, 1194],
 [119, 379, 712, 946, 1078],
 [108, 774, 794, 1026, 1045, 1075, 1187, 1197],
 [48,
  50,
  68,
  71,
  83,
  100,
  102,
  114,
  128,
  167,
  185,
  203,
  220,
  232,
  274,
  285,
  303,
  331,
  378,
  412,
  419,
  443,
  466,
  503,
  516,
  553,
  563,
  645,
  701,
  879,
  980,
  1013,
  1019,
  1029,
  1096,
  1108,
  1123,
  1153,
  1178,
  1185,
  1227],
 [431, 570, 759, 982],
 [22,
  37,
  162,
  233,
  349,
  372,
  399,
  489,
  532,
  671,
  952,
  979,
  996,
  1058,
  1066,
  1165,
  1175,
  1238],
 [104, 187, 248, 396, 541, 716, 797, 857

In [28]:
np.array(tar_set)[np.array(c[9])[:]]

array([[ 3,  0,  3,  3,  0,  1,  1,  0],
       [ 0,  2,  2, -2,  0,  1,  0,  0],
       [ 3,  1,  4,  2,  1,  1,  1,  1],
       [ 1,  2,  3, -1,  1,  1,  1,  0],
       [ 2,  1,  3,  1,  1,  1,  1,  0]])

In [16]:
c[0][0]

164